In [5]:
import streamlit as st
import pandas as pd

In [6]:
from Features import world_constructions as WCf

In [7]:
df= WCf.world_construction.init_luxury_gallery()

In [8]:
from DB_and_Azure.sql_db_functions import sql_db_functions as SQLf
from DB_and_Azure import Chroma_db_functions as Cf

In [9]:
conn, cursor = SQLf.connect_sql()

query = """
    SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand 
    FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id
    WHERE Brand = 'Gucci' or Brand = 'Prada'
    ;
"""
cursor.execute(query)
rows = cursor.fetchall()

SQLf.close_connection_db(conn=conn,cursor=cursor)

df = pd.DataFrame(rows, columns=['id', 'prod_id' , 'Detail', 'Summary', 'Brand'])

df['Detail'] = df['Detail'] = df['Detail'].str.replace('\n', ' / ').str.replace(r'\s+', ' ').str.replace('*', ' ')

df.prod_id = df.prod_id.astype(int)
df.prod_id.to_numpy

prod_ids = ",".join(df.prod_id.astype('string').tolist())

conn, cursor = SQLf.connect_sql()
query = f"SELECT Brand_id, base64 FROM product_img WHERE Brand_id in ({prod_ids}) ;"
cursor.execute(query)
rows_img = cursor.fetchall()

In [21]:
SQLf.close_connection_db(conn=conn,cursor=cursor)


df_image = pd.DataFrame(rows_img, columns=['prod_id', 'base64']).dropna()

In [23]:
df_image.shape

(101, 2)

In [24]:
df_image = df_image.groupby('prod_id')['base64'].apply(list).reset_index()
            
df = pd.merge(df,df_image, on='prod_id')

In [25]:
df

,id,prod_id,Detail,Summary,Brand,base64
0,161,1,Type: Shirt / Fit: Regular fit / ...,The item is a Gucci silk shirt featuring an op...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve...,The Gucci cotton Oxford GG shirt with an optic...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,163,3,Type: Cropped blouse / Fit: Regular fit ...,The Gucci top is a cropped blouse made of luxu...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve...,The garment in question is a Gucci Camicia in ...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve...,The garment in question is a Gucci cotton popl...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
5,166,6,Type: Shirt / Fit: Regular fit / Sleeve sty...,The Gucci silk shirt features the brand's icon...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
6,167,7,Type: Top / Fit: Slim fit / Sleeve style: S...,The top in question is a Gucci floral lace top...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
7,168,8,Type: Lace Top / Fit: Regular fit / Sle...,The garment in the images is a floral lace cot...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
8,169,9,Type: Shirt / Fit: Regular fit / ...,The Gucci shirt in question is a finely crafte...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
9,170,10,Type: Shirt / Fit: Regular fit / Sleeve...,The shirt displayed in the images is a luxurio...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
